Hello this note book is our first attempt at text generation.
THhs is an intro to nlp and the dangers of ai and ethics all in one

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import tensorflow.keras.utils as ku
from transformers import TFAutoModel, AutoTokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import re
import tensorflow as tf
import h5py

Reading and cleanning data so that we take only the unique tweets and remove all links in them. we also make sure that after we do that we remove any that are empty strings. additonally remove all double quotes

In [ ]:
df = pd.read_csv('../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv',index_col='Unnamed: 0')
df.describe()

In [ ]:
df['clean_tweets'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

In [ ]:
clean_tweets = np.array(df['clean_tweets'][df['clean_tweets'] != ''].unique())

In [ ]:
np.array([len(tweet) for tweet in clean_tweets]).argmin()

In [ ]:
clean_tweets = [x.replace('"','') for x in clean_tweets]

In [ ]:
bert = TFAutoModel.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
SEQ_LEN = 60

In [ ]:
def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=SEQ_LEN,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

Tokenizing the tweets. our model will take an input of (5=seq_len) word hashes and predict the next word we will try to do a different n-grams with padding and see if its better in the next model. 

In [ ]:
# tokenizer = Tokenizer()
# corpus = clean_tweets
# tokenizer.fit_on_texts(corpus)
# total_words = len(tokenizer.word_index) + 1
# print(total_words)

In [ ]:
token_list = tokenize(clean_tweets[100])
print(token_list)
print([clean_tweets[100]])

In [ ]:
# initialize two arrays for input tensors
Xids = np.zeros((len(clean_tweets), SEQ_LEN))
Xmask = np.zeros((len(clean_tweets), SEQ_LEN))

# loop through data and tokenize everything
for i, tweet in enumerate(clean_tweets):
    Xids[i, :], Xmask[i, :] = tokenize(tweet)

print(Xids, Xmask)


In [ ]:

# seq_len = 5
X_data = []
y_data = []
input_sequences = []
input_masks = []

# initialize two arrays for input tensors
# Xids = np.zeros((len(clean_tweets), SEQ_LEN))
# Xmask = np.zeros((len(clean_tweets), SEQ_LEN))

# loop through data and tokenize everything
# for i, tweet in enumerate(clean_tweets):
#     Xids[i, :], Xmask[i, :] = tokenize(tweet)

for tweet in clean_tweets:
    token_list, mask_list = tokenize(tweet)
    for i in range(1, SEQ_LEN):
        n_gram_sequence = token_list[0][:i+1]
        input_sequences.append(n_gram_sequence)
        input_masks.append(mask_list[0][:i+1])


In [ ]:
input_masks

In [ ]:
# pad sequences 
input_sequences = np.array(pad_sequences(input_sequences, maxlen=SEQ_LEN, padding='pre'))
input_masks = np.array(pad_sequences(input_masks, maxlen=SEQ_LEN, padding='pre'))

In [ ]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
pred_masks = input_masks[:,:-1]

In [ ]:
drop_list = []

for idx, val in enumerate(label):
    if val == 0:
        drop_list.append(idx)
        
print(len(drop_list))

In [ ]:
# keep_list = [i for i in range(label.shape[0]) if i not in drop_list]

In [ ]:
# hf = h5py.File('keeplist.h5', 'w')
# hf.create_dataset('keeplist', data=keep_list)
# hf.close()
# hf.create_dataset('dataset_2', data=d2)
hf = h5py.File('./keeplist.h5', 'r')
keep_list = np.array(hf.get('keeplist'))
hf.close()

In [ ]:
predictors = predictors[keep_list]
label = label[keep_list]
pred_masks = pred_masks[keep_list]
print(predictors.shape, label.shape)

In [ ]:
#del keep_list
del input_sequences
del input_masks
del clean_tweets
del Xids

In [ ]:
label = ku.to_categorical(label)
label.shape

In [ ]:
# create tensorflow dataset object
dataset = tf.data.Dataset.from_tensor_slices((predictors[:50000, :], pred_masks[:50000, :], label[:50000, :]))

In [ ]:
# restructure dataset format for BERT
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels


In [ ]:
dataset = dataset.map(map_func)  # apply the mapping function

# shuffle and batch the dataset
dataset = dataset.shuffle(100).batch(32)

In [ ]:
DS_LEN = len(list(dataset))  # get dataset length

SPLIT = 0.9  # we will create a 90-10 split

# create training-validation sets
train = dataset.take(round(DS_LEN*SPLIT))
val = dataset.skip(round(DS_LEN*SPLIT))

In [ ]:
# build the model
input_ids = tf.keras.layers.Input(shape=(59,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(59,), name='attention_mask', dtype='int32')

In [ ]:
embeddings = bert(input_ids, attention_mask=mask)[0]  # we only keep tensor 0 (last_hidden_state)

In [ ]:
#n_patterns = len(X_data) #157566 seq_length = 5
#pd.DataFrame(y_data).iloc[:,0].sort_values().unique()


In [ ]:
# X = np.reshape(X_data, (n_patterns, seq_len, 1))
# X = X/float(len(tokenizer.word_index.items()))
# y = np.asarray(pd.get_dummies(y_data))


In [ ]:
#X.shape, y.shape, len(tokenizer.word_index.items())

In [ ]:
# print(y[0][1529])
# len(X_data[0])

In [ ]:
# X.shape
# embeddings.shape

In [ ]:
# X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
X = tf.keras.layers.BatchNormalization()(embeddings)
print(X.shape)
X = tf.keras.layers.Bidirectional(LSTM(128))(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(label.shape[1], activation='softmax', name='outputs')(X)  # adjust based on number of sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [ ]:
model.summary()

In [ ]:
label.shape

In [ ]:
# model = Sequential()

# model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))

# model.add(Bidirectional(LSTM(256, return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# #model.add(LSTM(128))
# #model.add(Dropout(0.2))
# model.add(Dense(total_words, activation='softmax')) #outputs a on hot encoded row 
# model.summary()

In [ ]:
filename = "model_weights_saved.hdf5"
#model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:

tf.keras.utils.plot_model(model)

In [ ]:
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[acc])
filepath = "model_weights_saved_long.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]


In [ ]:
print(train)

In [ ]:
model.fit(dataset, epochs=100, batch_size=50)

In [ ]:
model.save('trump_model.h5')

In [ ]:
pattern = [6,
  4304,
  36,
  706,
  380,]
print(pattern)
#model.summary()
x = np.reshape(pattern, (1,len(pattern), 1))
print(x.shape)

In [ ]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(word) for word in list_of_indices]
    return(words)

# Creating texts 
my_texts = sequence_to_text(pattern)
my_texts
#tokenizer.sequences_to_texts_generator([pattern])

In [ ]:
seed_text = "This was the best"
next_words = 100

In [ ]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
    
print(seed_text)

In [ ]:
pattern = [6,1,4304,36,706,]
#pattern = [random.randint(1,total_words) for x in range(5)]
out = [sequence_to_text([value]) for value in pattern]
for i in range(50):
    x = np.reshape(pattern, (1,len(pattern), 1))
    x = x/float(len(tokenizer.word_index.items())) #cast as float so it doesn't do int divid 
#    print(x.shape)
    prediction = model.predict(x, verbose=0)
    #print(prediction[0][:100])
    index = np.argmax(prediction[0]) + 1 #it goes zero to 10383 so add 1 for offset
    #print(index)
    print(sequence_to_text([index]))
    
  

    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    seq_in = [sequence_to_text([value]) for value in pattern]
    #print(seq_in)
    out.append(sequence_to_text([index]))
print(out)


In [ ]:
print(prediction)

In [ ]:
sequence_to_text([2])

In [ ]:
#sequence_to_text(y_data)
#y_data